In [ ]:
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os

load_dotenv()

st.title("💬 Conversational Bot Powered by Multi-File Context")

# Setup only once per session
@st.cache_resource
def setup_chain():
    loader = TextLoader("mytext.txt")
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    vector_db = Chroma.from_documents(chunks, embedding=embeddings)

    retriever = vector_db.as_retriever()
    llm = ChatOpenAI(temperature=0)

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        output_key="answer"
    )


    return chain

# Initialize the chain
qa_chain = setup_chain()

# Display chat history if it exists
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

for i, (user_msg, ai_msg) in enumerate(st.session_state.chat_history):
    with st.chat_message("user"):
        st.markdown(user_msg)
    with st.chat_message("assistant"):
        st.markdown(ai_msg)

# Input box (chat-style)
user_input = st.chat_input("Ask a question about the document")

if user_input:
    with st.chat_message("user"):
        st.markdown(user_input)

    with st.spinner("Thinking..."):
        result = qa_chain({"question": user_input})
        answer = result["answer"]

    with st.chat_message("assistant"):
        st.markdown(answer)

    # Save to chat history
    st.session_state.chat_history.append((user_input, answer))


In [ ]:
__import__("pysqlite3")
import sys

sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import pysqlite3